# Tutorial 3: Baseline, LR with all features (RG)

---

first, let us call the data

In [1]:
%store -r df_input_RG
%store -r y_RG
%store -r df_RG

In [2]:
y_RG = y_RG.map({'S': 1, 'B': 0})

calling some packages

In [3]:
# Python packages 
import pandas as pd # for importing data into data frame format
import seaborn as sns # For drawing useful graphs, such as bar graphs
import numpy as np
import matplotlib.pyplot as plt

---

<b><i> Data splitting </i></b> 

In [4]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, Ytrain, Ytest = train_test_split( df_input_RG, y_RG, test_size = 0.3, random_state=3, stratify=y_RG) # train and valid sets

In [5]:
print(Xtrain.shape)
print(Xtest.shape)

(393, 2074)
(169, 2074)


---

<b><i> training LR </i></b> 

In [6]:
from sklearn.linear_model import LogisticRegression

In [7]:
# define the hyper-parameters
solvers = ['newton-cg', 'liblinear']
penalty = ['l2']
c_values = [1000, 100, 10, 1.0, 0.1, 0.01, 0.001]
lr_par = dict(solver=solvers,penalty=penalty,C=c_values)

In [8]:
lr = LogisticRegression() #defining the model

In [9]:
models = [[lr, 'lr', 14]]
par = [lr_par]

In [10]:
from source.ml_acc import get_accuracy_ml

In [11]:
num_ml_tools = len(par)
ml_dicts = {}

for m, par in zip(models, par):
    # for s in splits:
    #     X_train, X_test, y_train, y_test = train_test_split(df, y, test_size= s, random_state=1, stratify = y, shuffle = True)
    key0 = str(m[1])
    ml_dicts[key0] = {}
    for i in range(1):
        xtr =  Xtrain
        xte =  Xtest
        results = get_accuracy_ml (m[0], m[2], par, xtr.values, np.array(Ytrain), xte.values, np.array(Ytest)) # to get the accuracies for the ml model

        key = str(m[1])+", all"
        ml_dicts[key0][key] = {}

        ml_dicts[key0][key]['tot_acc'] = results[0]
        ml_dicts[key0][key]['jack_train'] = results[1]
        ml_dicts[key0][key]['jack_test'] = results[2]

In [12]:
print(ml_dicts)

{'lr': {'lr, all': {'tot_acc': 0.8421052631578947, 'jack_train': [0.8421052631578947, 0.8333333333333334, 0.8421052631578947, 0.8421052631578947, 0.8421052631578947, 0.8421052631578947, 0.8421052631578947, 0.8350515463917526, 0.8421052631578947, 0.8421052631578947, 0.8333333333333334, 0.8333333333333334, 0.8421052631578947, 0.8421052631578947, 0.8421052631578947, 0.8421052631578947, 0.8421052631578947, 0.8421052631578947, 0.8421052631578947, 0.8421052631578947, 0.8421052631578947, 0.8421052631578947, 0.8421052631578947, 0.8421052631578947, 0.8421052631578947, 0.8350515463917526, 0.8333333333333334, 0.8421052631578947, 0.8421052631578947, 0.8421052631578947, 0.8421052631578947, 0.8421052631578947, 0.8350515463917526, 0.8350515463917526, 0.8350515463917526, 0.8421052631578947, 0.8350515463917526, 0.8421052631578947, 0.8421052631578947, 0.8421052631578947, 0.8421052631578947, 0.8421052631578947, 0.8421052631578947, 0.8421052631578947, 0.8247422680412371, 0.8421052631578947, 0.842105263157

---

<b><i> save the Dict into a file </i></b> 

In [13]:
import json

In [14]:
with open('base_rg.txt', 'w') as file:
     file.write(json.dumps(ml_dicts)) # use `json.loads` to do the reverse

---

In [15]:
from source.calculate_jack import jack_SD # importing the baseline code from source.basline file
arr_all = []
for m, d in zip (models, ml_dicts.keys()):
    acc_arr = [] 
    sd_arr = [] 

    # print(ml_dicts[d])
    for key in ml_dicts[d].keys():
        acc_arr.append(ml_dicts[d][key][ 'tot_acc' ]) # append total accuracy to an array
        sd_train = jack_SD(np.zeros( len(ml_dicts[d][key][ 'jack_train' ]) ), ml_dicts[d][key][ 'jack_train' ])[0]
        sd_test = jack_SD(np.zeros( len(ml_dicts[d][key][ 'jack_test' ]) ), ml_dicts[d][key][ 'jack_test' ])[0]
        sd = np.sqrt( np.array((sd_train**2)) + np.array((sd_test**2)))
        sd_arr.append(sd) # append sd_arr to an array
    arr_all.append([ list(ml_dicts[d].keys()), acc_arr, sd_arr])  

In [16]:
print(arr_all)

[[['lr, all'], [0.8421052631578947], [0.05893620563646332]]]


In [19]:
print(arr_all)

[[['lr, all'], [0.81], [0.04421751306500327]]]


---